In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import tensorflow as tf
#import  format_output
from datetime_extension import timedate_process
from produce_rnn_flow import aggregate_time
from math import sqrt,floor

In [2]:
#read block data
df=pd.read_csv("blocks.csv",header=0)
#print(df.shape)
df=timedate_process(df)
df=aggregate_time(df)
df.head()


,Level,latBlock,lngBlock,month,day,h2,h23,h12,h19,h1,...,h15_neighbor2,h16_neighbor2,h17_neighbor2,h18_neighbor2,h19_neighbor2,h20_neighbor2,h21_neighbor2,h22_neighbor2,h23_neighbor2,days
0,0,0,20,4,1,0,0,0,0,0,...,24.5,80.0,20.0,3.0,1.0,1.0,1.0,1.0,0.0,0
1,0,0,20,4,2,0,0,0,0,0,...,16.0,16.5,35.5,2.5,1.5,1.5,2.0,1.5,0.5,1
2,0,0,20,4,3,0,0,0,0,0,...,34.0,4.5,7.5,8.5,1.5,1.5,1.5,1.5,0.5,2
3,0,0,20,4,5,0,0,1,0,0,...,6.5,30.5,29.5,0.0,4.5,44.5,21.5,14.5,31.5,4
4,0,0,20,4,6,0,0,0,0,0,...,16.5,28.5,9.0,12.5,5.5,9.0,1.5,1.5,0.5,5


In [3]:
#print(max(df[(level=='0')].latBlock))
#df=df[(df.Level==0) & (df.latBlock==0) & (df.lngBlock==20)]
df=df[(df.Level==1)]
df.drop(['Level','latBlock','lngBlock','month','day'], axis=1,inplace=True)
print(df)

        h2  h23  h12  h19  h1  h10  h14  h8  h21  h22  ...   h15_neighbor2  \
62148    0    0    0    0   0    0    0   0    0    0  ...             7.5   
62149    0    0    0    0   0    0    0   0    0    0  ...             5.0   
62150    0    0    0    0   0    0    1   0    0    0  ...             3.0   
62151    0    0    1    0   0    0    0   0    0    0  ...             3.5   
62152    0    0    0    1   0    0    0   0    0    0  ...            20.0   
62153    0    0    0    0   0    0    0   0    0    1  ...            30.0   
62154    0    0    0    0   0    0    0   0    0    0  ...            20.0   
62155    0    0    0    0   0    0    0   0    0    0  ...            86.5   
62156    0    0    1    0   0    0    0   0    0    0  ...            11.5   
62157    0    0    0    0   0    0    0   0    0    0  ...            37.5   
62158    0    0    0    1   0    0    0   0    0    0  ...            21.0   
62159    0    0    0    0   0    0    0   0    0    0  ...      

In [4]:
data=[]
for i in range(0,len(df)):
    data.append(df.iloc[i].tolist())
data=np.array(data)

In [5]:
# define parameter
rnn_unit = 10  # hidden layer units
input_size = 73
output_size = 24
lr = 0.0006  # learning rate
N = 500 #ecope
N_model = 50 #each N time save model
length = 0
# ——————————————————input data——————————————————————
length=range(len(data))
#print(data)
# get train set
def get_train_data(batch_size=85, time_step=1, train_begin=0, train_end=length):
    batch_index = []
    data_train = data[train_begin:train_end]
    normalized_train_data = data_train
    #print(normalized_train_data)#
    train_x, train_y = [], []  # train set
    for i in range(len(normalized_train_data) - time_step):
        if i % batch_size == 0:
            batch_index.append(i)
        x = normalized_train_data[i:i + time_step, :73]
        y = normalized_train_data[i:i + time_step, 0:24]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    batch_index.append((len(normalized_train_data) - time_step))
    #print(train_y)
    return batch_index, train_x, train_y


# test set
def get_test_data(time_step=1, test_begin=0):
    data_test = data[test_begin:]
    mean = np.mean(data_test, axis=0)
    std = np.std(data_test, axis=0)

    normalized_test_data = data_test  # normalized
    size = (len(normalized_test_data) + time_step - 1) // time_step  # 
    test_x, test_y = [], []
    for i in range(size - 1):
        x = normalized_test_data[i * time_step:(i + 1) * time_step, :73]
        y = normalized_test_data[i * time_step:(i + 1) * time_step, 0:24]
        test_x.append(x.tolist())
        test_y.extend(y)
    test_x.append((normalized_test_data[(i + 1) * time_step:, :73]).tolist())
    test_y.extend((normalized_test_data[(i + 1) * time_step:, 0:24]).tolist())
    return mean, std, test_x, test_y




weights = {
    'in': tf.Variable(tf.random_normal([input_size, rnn_unit])),
    #'out': tf.Variable(tf.random_normal([rnn_unit, 1]))
    'out': tf.Variable(tf.random_normal([rnn_unit, output_size]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[rnn_unit, ])),
    #'out': tf.Variable(tf.constant(0.1, shape=[1, ]))
    'out': tf.Variable(tf.constant(0.1, shape=[output_size, ]))
}



def lstm(X):
    batch_size = tf.shape(X)[0]
    time_step = tf.shape(X)[1]
    w_in = weights['in']
    b_in = biases['in']
    input = tf.reshape(X, [-1, input_size])  
    input_rnn = tf.matmul(input, w_in) + b_in
    input_rnn = tf.reshape(input_rnn, [-1, time_step, rnn_unit]) 
    cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state = cell.zero_state(batch_size, dtype=tf.float32)
    output_rnn, final_states = tf.nn.dynamic_rnn(cell, input_rnn, initial_state=init_state,
                                                 dtype=tf.float32)  
    output = tf.reshape(output_rnn, [-1, rnn_unit])  
    w_out = weights['out']
    b_out = biases['out']
    pred = tf.matmul(output, w_out) + b_out
    return pred, final_states



def train_lstm(batch_size=2, time_step=1, train_begin=0, train_end=10):
    X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
    Y = tf.placeholder(tf.float32, shape=[None, time_step, output_size])
    batch_index, train_x, train_y = get_train_data(batch_size, time_step, train_begin, train_end)
    with tf.variable_scope("sec_lstm"):
        pred, _ = lstm(X)

    loss = tf.reduce_mean(tf.square(tf.reshape(pred, [-1,output_size]) - tf.reshape(Y, [-1,output_size])))
    train_op = tf.train.AdamOptimizer(lr).minimize(loss)
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
    with tf.Session() as sess:
        try:
            module_file = tf.train.latest_checkpoint('.')
            saver.restore(sess, module_file)
        except:
            sess.run(tf.global_variables_initializer())
        # 重复训练N次
        for i in range(N):
            for step in range(len(batch_index) - 1):
                _, loss_ = sess.run([train_op, loss], feed_dict={X: train_x[batch_index[step]:batch_index[step + 1]],
                                                                 Y: train_y[batch_index[step]:batch_index[step + 1]]})
            #print(i, loss_)
            if i % N_model==0:
                print("save model：", saver.save(sess, '.\prediction.model', global_step=i))

train_lstm()






INFO:tensorflow:Restoring parameters from .\prediction.model-450
save model： .\prediction.model-0
save model： .\prediction.model-50
save model： .\prediction.model-100
save model： .\prediction.model-150
save model： .\prediction.model-200
save model： .\prediction.model-250
save model： .\prediction.model-300
save model： .\prediction.model-350
save model： .\prediction.model-400
save model： .\prediction.model-450


In [6]:
def prediction(time_step=1,time_select=0):
    X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
    Y = tf.placeholder(tf.float32, shape=[None,time_step,output_size])
    mean, std, test_x, test_y = get_test_data(time_step)
    with tf.variable_scope("sec_lstm", reuse=True):
        pred, _ = lstm(X)
    saver = tf.train.Saver(tf.global_variables())
    with tf.Session() as sess:
        module_file = tf.train.latest_checkpoint('.')
        saver.restore(sess, module_file)
        
        
        test_predict = []
        for step in range(len(test_x) - 1):
            prob = sess.run(pred, feed_dict={X: [test_x[step]]})
            predict = prob.reshape((-1,1))
            test_predict.extend(predict)
        predict_test=[]
        for i in range(len(test_predict)):
            if i%24==time_select:
                predict_test.append(test_predict[i])
        #print(predict_test)
        y_test=[]
        test_y=np.array(test_y).reshape(-1,1)
        for i in range(len(test_y)):
            if i%24==time_select:
                y_test.append(test_y[i])
        test_y=np.array(y_test)
        test_predict=np.array(predict_test)
        test_y=np.nan_to_num(test_y)
        test_predict=np.nan_to_num(test_predict)
        test_y_result=[]
        test_predict_result=[]
        for i in test_y:
            test_y_result.append(i[0])
        for i in test_predict:
            test_predict_result.append(i[0])    
        test_predict_result=np.array(test_predict_result)
        test_y_result=np.array(test_y_result)
        result_list=[]
        #print(test_predict)
        for j in range(0,floor(min(len(test_y_result),len(test_predict_result))/85)):
            target=[]
            prediction=[]
            target=test_y_result[j*85:(j+1)*85]
            prediction=test_predict_result[j*85:(j+1)*85]
            error = []
            #print(min(len(target),len(prediction)))
            for i in range(min(len(target),len(prediction))):
                error.append(target[i] - prediction[i])
            squaredError = []
            absError = []
            #print(error)
            for val in error:
                squaredError.append(val * val)#target-prediction之差平方 
                absError.append(abs(val))#误差绝对值
            #print(j)
            result_list.append(sqrt(sum(squaredError) / len(squaredError)))
        return result_list
        #print("RMSE = ", sqrt(sum(squaredError) / len(squaredError)))#均方根误差RMSE
        #plt.figure()
        #plt.plot(list(range(len(test_predict))), test_predict, color='b')
        #plt.plot(list(range(len(test_y))), test_y,  color='r')
        #plt.show()
RMSE_result=[]
for i in range(0,24):
    RMSE_result.append(prediction(1,i))

INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring parameters from .\prediction.model-450
INFO:tensorflow:Restoring

In [7]:
rmseBlockRulst=[]
avg=0
for j in range(0,len(RMSE_result[0])):
    avg=0
    for i in RMSE_result:
        avg=avg+i[j]
    rmseBlockRulst.append(avg/24)
print(rmseBlockRulst)
print(max(rmseBlockRulst))

[0.43497822562568705, 2.4322270911638246, 5.216338202468729, 59.407799623044696, 14.494535628416997, 2.6128158678640276, 4.2523363712360425, 1.1465689757211805, 0.6810128318304963, 0.6305242848560654, 22.461244708158976, 4.3873885798928685, 0.475995582391795, 0.7903664825829121, 1.8252821962538952, 3.74050230839643, 10.541359333000196, 27.25978093197952, 22.614176529973225, 52.91881160701624, 77.80126223822576, 20.596435982865554, 55.92454429130899, 27.169642841154186, 20.99503821918304, 6.415505623133946, 10.321368506779866, 7.093219238105303, 57.85830887468313, 74.51855720989094, 92.99893070049832, 53.35698336277667, 6.107517217026646, 6.858607714548619, 0.8368321378114633, 0.646883477709796, 0.5293229249797762, 0.6040194045619144, 0.6205956039160944, 0.7606717748358318, 1.572181000948256, 4.680173415296731, 3.248383528889439, 6.126386148319534, 11.756299250288082, 41.30712593418392, 45.05687649961902, 56.36239903537089, 77.55033440417007, 58.41204468856849, 106.45221588832072, 111.0

In [8]:
array_rmse=np.array(rmseBlockRulst)
print(array_rmse.mean())

24.800465655265302
